In [1]:
import polars as pl
import altair

In [67]:
stations = pl.read_csv("data/current_bluebikes_stations.csv", skip_rows=1)
cambridge_stations = stations.filter(pl.col("District") == "Cambridge")
display(cambridge_stations)
cambridge_station_ids = {s for s in cambridge_stations.to_dict()["Name"]}

Number,Name,Latitude,Longitude,District,Public,Total docks,Deployment Year
str,str,f64,f64,str,str,i64,str
"""M32026""","""359 Broadway -…",42.370803,-71.104412,"""Cambridge""","""Yes""",23,"""2013"""
"""M32054""","""699 Mt Auburn …",42.375002,-71.148716,"""Cambridge""","""Yes""",25,"""2018"""
"""M32060""","""700 Huron Ave""",42.380788,-71.154129,"""Cambridge""","""Yes""",19,"""2018"""
"""M32064""","""75 Binney St""",42.365507,-71.080138,"""Cambridge""","""Yes""",19,"""2019"""
"""M32058""","""84 Cambridgepa…",42.3936,-71.143941,"""Cambridge""","""Yes""",27,"""2018"""
"""M32073""","""955 Mass Ave""",42.368952,-71.109988,"""Cambridge""","""Yes""",19,"""2021"""
"""M32046""","""Alewife MBTA a…",42.395588,-71.142606,"""Cambridge""","""Yes""",19,"""2016"""
"""M32033""","""Alewife Statio…",42.396105,-71.139459,"""Cambridge""","""Yes""",23,"""2014"""
"""M32059""","""Ames St at Bro…",42.363693,-71.087567,"""Cambridge""","""Yes""",19,"""2018"""


In [61]:
rides = pl.read_csv("data/20*.csv", columns=["starttime", "start station id", "end station id", "start station name", "end station name"]).with_columns(pl.col("starttime").str.slice(0, 10).str.strptime(pl.Date, fmt="%Y-%m-%d", strict=False)).sort("starttime")
rides 

starttime,start station id,end station id,start station name,end station name
date,i64,i64,str,str
2016-01-01,36,19,"""Boston Public …","""Buswell St. at…"
2016-01-01,36,19,"""Boston Public …","""Buswell St. at…"
2016-01-01,36,19,"""Boston Public …","""Buswell St. at…"
2016-01-01,36,19,"""Boston Public …","""Buswell St. at…"
2016-01-01,36,67,"""Boston Public …","""MIT at Mass Av…"
2016-01-01,110,88,"""Harvard Univer…","""Inman Square a…"
2016-01-01,107,176,"""Ames St at Mai…","""Lesley Univers…"
2016-01-01,141,90,"""Kendall Street…","""Lechmere Stati…"
2016-01-01,178,80,"""MIT Pacific St…","""MIT Stata Cent…"


In [68]:
cambridge_rides = rides.filter(pl.col("start station name").is_in(cambridge_station_ids) | pl.col("end station name").is_in(cambridge_station_ids))
cambridge_rides

starttime,start station id,end station id,start station name,end station name
date,i64,i64,str,str
2016-01-01,36,67,"""Boston Public …","""MIT at Mass Av…"
2016-01-01,107,176,"""Ames St at Mai…","""Lesley Univers…"
2016-01-01,141,90,"""Kendall Street…","""Lechmere Stati…"
2016-01-01,178,80,"""MIT Pacific St…","""MIT Stata Cent…"
2016-01-01,68,178,"""Central Square…","""MIT Pacific St…"
2016-01-01,72,178,"""One Broadway /…","""MIT Pacific St…"
2016-01-01,68,177,"""Central Square…","""University Par…"
2016-01-01,68,177,"""Central Square…","""University Par…"
2016-01-01,68,88,"""Central Square…","""Inman Square a…"


In [69]:
def by_month(df):
    return df.groupby_dynamic("starttime", every="1mo").agg(pl.count())

def by_year(df):
    return df.groupby_dynamic("starttime", every="1y").agg(pl.count())

monthly = by_month(cambridge_rides)
display(monthly.select([pl.col("count").sum()]))
yearly = by_year(cambridge_rides)
display(yearly.select([pl.col("count").sum()]))


count
u32
6983372


count
u32
6983372


In [76]:
def linechart(df, title):
    return altair.Chart(df.to_pandas()).mark_line().encode(x="starttime:T", y="count:Q").properties(title=title)

display(linechart(monthly, 'Monthly Bluebikes rides starting/ending in Cambridge'))
display(linechart(yearly, 'Yearly Bluebikes rides starting/ending in Cambridge'))

alt.Chart(...)

alt.Chart(...)